In [9]:
import os
import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from func import *
import json
import ast

In [10]:
df = pd.read_csv("merged_df.csv")
df

,image_id,bbox_id,keypoints,center,scale,area,score,bbox,width,height,bbox_size,x,y
0,139,0,"[[429.280517578125, 169.61883544921875, 0.8657...","[438.82501220703125, 226.11500549316406]","[0.6422343850135803, 0.8563125133514404]",21998.134766,0.763357,"[412.8, 157.61, 53.05, 138.01]",53.05,138.01,7321.4305,412.80,157.61
1,785,0,"[[367.1922302246094, 80.87351989746094, 0.9483...","[389.6400146484375, 217.57000732421875]","[1.6203750371932983, 2.1605000495910645]",140032.812500,0.902015,"[280.79, 44.73, 218.7, 346.68]",218.70,346.68,75818.9160,280.79,44.73
2,872,0,"[[214.49267578125, 184.88751220703125, 0.79049...","[290.7349853515625, 328.8450012207031]","[2.1391406059265137, 2.852187395095825]",244049.203125,0.664442,"[145.26, 100.67, 291.95, 457.35]",291.95,457.35,133523.3325,145.26,100.67
3,885,2,"[[632.353759765625, 46.84330749511719, 0.96756...","[616.864990234375, 139.02000427246094]","[1.0616250038146973, 1.4155000448226929]",60109.207031,0.805771,"[595.68, 25.78, 43.37, 227.48]",43.37,227.48,9865.8076,595.68,25.78
4,885,0,"[[373.1753234863281, 217.2002716064453, 0.9457...","[346.8550109863281, 293.6000061035156]","[0.9713437557220459, 1.2951250076293945]",50320.460938,0.843057,"[277.31, 189.99, 140.09, 208.22]",140.09,208.22,29169.5398,277.31,189.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,580294,0,"[[587.7160034179688, 73.6966552734375, 0.31208...","[525.0700073242188, 220.07000732421875]","[2.034562587738037, 2.712750196456909]",220770.406250,0.383998,"[411.14, 3.05, 228.86, 435.04]",228.86,435.04,99563.2544,411.14,3.05
6330,581062,0,"[[163.4222869873047, 66.69705200195312, 0.9830...","[162.58999633789062, 107.5250015258789]","[0.5272969007492065, 0.7030624747276306]",14828.906250,0.896881,"[131.74, 51.28, 62.7, 113.49]",62.70,113.49,7115.8230,131.74,51.28
6331,581206,0,"[[347.857177734375, 109.86737060546875, 0.1125...","[234.26499938964844, 109.66000366210938]","[2.928312301635742, 3.904416561126709]",457334.062500,0.339190,"[0, 2.53, 469.53, 215.26]",469.53,215.26,101071.0278,0.00,2.53
6332,581317,0,"[[501.866455078125, 118.10910034179688, 0.9580...","[511.06500244140625, 204.46499633789062]","[1.3925156593322754, 1.8566875457763672]",103418.664062,0.849606,"[409.85, 55.93, 203.43, 298.07]",203.43,298.07,60636.3801,409.85,55.93


In [11]:
df.columns

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'area', 'score',
       'bbox', 'width', 'height', 'bbox_size', 'x', 'y'],
      dtype='object')

In [12]:
df.drop(columns=['bbox_id','keypoints','area'], inplace=True)

In [13]:
df["image_id"] = df["image_id"].astype(str).apply(lambda x: x.rjust(12, '0') + ".jpg")
df['bbox'] = df['bbox'].apply(ast.literal_eval)

In [14]:
df['image_file'] = str('val2017/')+df['image_id']
df.drop(columns='image_id', inplace=True)

In [15]:
df.columns

Index(['center', 'scale', 'score', 'bbox', 'width', 'height', 'bbox_size', 'x',
       'y', 'image_file'],
      dtype='object')

## metric 계산 적용

In [16]:
entropy = []
laplacian = []
brightness = []
B_sd = []
c_moment = []
contra = []
img_size = []
edge_intensity = []

for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    height, width= image.shape
    img_size_value = width * height

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    entropy_value = calculate_image_entropy_(cropped_image)
    laplacian_value = calculate_laplacian_variance_(cropped_image)
    brightness_value = np.mean(cropped_image)
    sd_value = np.std(cropped_image)
    c_moment_value = moment(cropped_image)
    contrast_value = calculate_contrast(cropped_image)
    edge_value = edge(cropped_image)

    entropy.append(entropy_value)
    laplacian.append(laplacian_value)
    brightness.append(brightness_value)
    B_sd.append(sd_value)
    c_moment.append(c_moment_value)
    contra.append(contrast_value)
    img_size.append(img_size_value)
    edge_intensity.append(edge_value)

In [17]:
red = []
green = []
blue = []
color_var = []

# Not Grayscale
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    
    # 이미지 배열에서 각 채널의 값 가져오기
    red_channel = cropped_image[:, :, 0]
    green_channel = cropped_image[:, :, 1]
    blue_channel = cropped_image[:, :, 2]

    # 각 채널별 평균과 분산 계산
    red_mean, red_variance = np.mean(red_channel), np.var(red_channel)
    green_mean, green_variance = np.mean(green_channel), np.var(green_channel)
    blue_mean, blue_variance = np.mean(blue_channel), np.var(blue_channel)
    variance = np.mean([red_variance,green_variance,blue_variance])

    red.append(red_mean)
    green.append(green_mean)
    blue.append(blue_mean)
    color_var.append(variance)

In [18]:
df['entropy'] = entropy
df['laplacian'] = laplacian
df['brightness'] = brightness
df['B_sd'] = B_sd
df['red'] = red
df['blue'] = blue
df['green'] = green
df['color_var'] = color_var
df['c_moment'] = c_moment
df['contrast'] = contra
df['img_size'] = img_size
df['edge_intensity'] = edge_intensity

In [19]:
# 새로운 열 생성
df[['c_moment_x', 'c_moment_y']] = pd.DataFrame(df['c_moment'].tolist(), index=df.index)

# 기존 열 삭제
# df = df.drop('c_moment', axis=1)

In [20]:
df['bbox_prop'] = df['bbox_size']/df['img_size'].values

In [21]:
df = df[['image_file'] + [col for col in df.columns if col != 'image_file']]

In [22]:
tmp = df.sort_values(by='score', ascending=False).reset_index(drop=True)

In [23]:
tmp.columns

Index(['image_file', 'center', 'scale', 'score', 'bbox', 'width', 'height',
       'bbox_size', 'x', 'y', 'entropy', 'laplacian', 'brightness', 'B_sd',
       'red', 'blue', 'green', 'color_var', 'c_moment', 'contrast', 'img_size',
       'edge_intensity', 'c_moment_x', 'c_moment_y', 'bbox_prop'],
      dtype='object')

In [24]:
tmp.to_csv("refine_merged_df.csv", index=False)